In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('existing_stations.csv')
df.head()

,Unnamed: 0,Station Name,Street Address,City,State,ZIP,Latitude,Longitude,EV Network,EV Connector Types
0,0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,34.248319,-118.387971,SHELL_RECHARGE,"CHADEMO,J1772,J1772COMBO"
1,1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,34.040539,-118.271387,Non-Networked,J1772
2,2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,34.059133,-118.248589,Non-Networked,"CHADEMO,J1772,J1772COMBO"
3,3,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,33.759802,-118.096665,Non-Networked,"CHADEMO,J1772,J1772COMBO"
4,4,LADWP - Harbor Generating Station,161 N Island Ave,Wilmington,CA,90744,33.770508,-118.265628,Non-Networked,J1772


In [5]:
df['State'].value_counts()

State
CA    108
Name: count, dtype: int64

In [6]:
df = df[df['State'] == 'CA']

In [7]:
df.shape

(108, 10)

In [15]:
import time
import random
import numpy as np
import pandas as pd
import folium
from folium import plugins
from branca.element import Template, MacroElement
from geopy.distance import geodesic
from pyswarm import pso
from deap import base, creator, tools
import geopandas as gpd
from shapely.geometry import Point

# ----------------------------------------------------------------------
# 1. Data and Threshold
# ----------------------------------------------------------------------

# Replace this with your actual station data
df = pd.read_csv('existing_stations.csv')
# Ensure columns: ['Station Name','Latitude','Longitude']

existing_stations = df[['Station Name', 'Latitude', 'Longitude']]
existing_locations = [(row['Latitude'], row['Longitude']) for _, row in existing_stations.iterrows()]

def calculate_distance(p1, p2):
    """Geodesic distance (km)."""
    return geodesic(p1, p2).kilometers

def calculate_threshold_distance(locations):
    """
    Calculate a threshold distance based on each existing station's average
    distance to its 3 closest neighbors, and then averaging those values.
    """
    avg_distances = []
    for i, station in enumerate(locations):
        distances = []
        for j, other_station in enumerate(locations):
            if i != j:
                distances.append(calculate_distance(station, other_station))
        distances.sort()
        closest_3 = distances[:3] if len(distances) >= 3 else distances
        avg_distances.append(sum(closest_3)/len(closest_3))
    return sum(avg_distances)/len(avg_distances)

threshold_distance = calculate_threshold_distance(existing_locations)
print(f"Threshold distance: {threshold_distance:.2f} km")

# ----------------------------------------------------------------------
# 2. Load California Boundary
# ----------------------------------------------------------------------

# URL to the GeoJSON file containing U.S. states
usa_states_url = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'

# Load the GeoJSON data into a GeoDataFrame
usa_states = gpd.read_file(usa_states_url)

# Filter for California
california = usa_states[usa_states['name'] == 'California']
if california.empty:
    raise ValueError("California not found in the dataset. Please check the state name or dataset.")

california_polygon = california.geometry.unary_union

def is_within_california(lat, lon, california_polygon):
    point = Point(lon, lat)  # Note: Point takes (longitude, latitude)
    return california_polygon.contains(point)

# ----------------------------------------------------------------------
# 3. PSO Setup with California Constraints
# ----------------------------------------------------------------------

N_NEW_STATIONS = 10

# Tight bounding box for California
lat_min, lat_max = 32.5343, 42.0095
lon_min, lon_max = -124.4096, -114.1312

lb = [lat_min, lon_min] * N_NEW_STATIONS
ub = [lat_max, lon_max] * N_NEW_STATIONS

def fitness_function_pso(new_station_locations, california_polygon):
    penalty_outside = 1e8  # Higher penalty for being outside California
    penalty_threshold = 1e6  # Penalty for violating threshold distance
    total_distance = 0
    num_new_stations = len(new_station_locations) // 2

    for i in range(num_new_stations):
        lat = new_station_locations[2 * i]
        lon = new_station_locations[2 * i + 1]
        new_coord = (lat, lon)

        # Check if within California
        if not is_within_california(lat, lon, california_polygon):
            return penalty_outside  # Constraint violation

        # Calculate distance to existing stations
        distances_to_existing = [calculate_distance(new_coord, e) for e in existing_locations]
        min_dist = min(distances_to_existing)

        if min_dist < threshold_distance:
            return penalty_threshold  # Constraint violation

        total_distance += min_dist

    return total_distance / num_new_stations

def run_pso_optimization(california_polygon):
    start = time.time()
    def fitness_wrapper(new_station_locations):
        return fitness_function_pso(new_station_locations, california_polygon)

    best_locations, best_value = pso(
        fitness_wrapper,
        lb=lb,
        ub=ub,
        swarmsize=50,
        maxiter=50
    )
    end = time.time()
    duration = end - start

    best_coords = [(best_locations[2 * i], best_locations[2 * i + 1]) for i in range(N_NEW_STATIONS)]

    return best_coords, best_value, duration

# ----------------------------------------------------------------------
# 4. GA Setup with California Constraints
# ----------------------------------------------------------------------

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()

def generate_location(california_polygon):
    attempts = 0
    max_attempts = 1000

    while attempts < max_attempts:
        lat = random.uniform(lat_min, lat_max)
        lon = random.uniform(lon_min, lon_max)
        if is_within_california(lat, lon, california_polygon):
            distances = [calculate_distance((lat, lon), e) for e in existing_locations]
            if min(distances) >= threshold_distance:
                return [lat, lon]
        attempts += 1

    raise ValueError("Exceeded maximum attempts to generate a valid location within California.")

def init_individual():
    coords = []
    for _ in range(N_NEW_STATIONS):
        coords.extend(generate_location(california_polygon))
    return creator.Individual(coords)

toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def fitness_function_ga(individual, california_polygon):
    penalty_outside = 1e8  # Higher penalty for being outside California
    penalty_threshold = 1e6  # Penalty for violating threshold distance
    total_distance = 0
    num_new_stations = len(individual) // 2

    for i in range(num_new_stations):
        lat = individual[2 * i]
        lon = individual[2 * i + 1]
        new_coord = (lat, lon)

        if not is_within_california(lat, lon, california_polygon):
            return (penalty_outside,)

        distances_to_existing = [calculate_distance(new_coord, e) for e in existing_locations]
        min_dist = min(distances_to_existing)

        if min_dist < threshold_distance:
            return (penalty_threshold,)

        total_distance += min_dist

    return (total_distance / num_new_stations,)

def constrained_mutate(individual, california_polygon):
    mutation_probability = 0.3  # Increased mutation rate to 0.3

    for i in range(0, len(individual), 2):
        if random.random() < mutation_probability:
            individual[i] += random.gauss(0, 0.01)  # Mutate latitude
            individual[i + 1] += random.gauss(0, 0.01)  # Mutate longitude

            # Clamp to bounding box
            individual[i] = max(min(individual[i], lat_max), lat_min)
            individual[i + 1] = max(min(individual[i + 1], lon_max), lon_min)

            # Check if within California
            if not is_within_california(individual[i], individual[i + 1], california_polygon):
                new_coord = generate_location(california_polygon)
                individual[i], individual[i + 1] = new_coord
            else:
                distances = [calculate_distance((individual[i], individual[i + 1]), e) for e in existing_locations]
                if min(distances) < threshold_distance:
                    new_coord = generate_location(california_polygon)
                    individual[i], individual[i + 1] = new_coord

    return (individual,)

toolbox.register("evaluate", fitness_function_ga, california_polygon=california_polygon)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", constrained_mutate, california_polygon=california_polygon)

def initialize_population_from_pso(pso_coords):
    ind_data = []
    for (lat, lon) in pso_coords:
        ind_data.append(lat)
        ind_data.append(lon)
    return creator.Individual(ind_data)

def run_ga_optimization(california_polygon):
    pop_size = 50  # Increased population size
    n_gen = 50  # Increased number of generations
    cx_prob = 0.5
    mut_prob = 0.3  # Increased mutation rate to 0.3

    population = toolbox.population(n=pop_size)
    for ind in population:
        ind.fitness.values = toolbox.evaluate(ind)

    start = time.time()
    for gen in range(n_gen):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cx_prob:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mut_prob:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)

        population[:] = offspring
    end = time.time()
    duration = end - start

    best_ind = min(population, key=lambda ind: ind.fitness.values[0])
    best_fitness = best_ind.fitness.values[0]

    best_coords = []
    for i in range(N_NEW_STATIONS):
        lat = best_ind[2 * i]
        lon = best_ind[2 * i + 1]
        best_coords.append((lat, lon))

    return best_coords, best_fitness, duration

# ----------------------------------------------------------------------
# 5. Main Function
# ----------------------------------------------------------------------

def main():
    best_fitness = float('inf')
    best_coords = None
    results = []

    # Run 10 simulations
    for run_idx in range(1, 11):
        print(f"\n--- Hybrid Simulation #{run_idx} ---")
        try:
            # Seed GA with best PSO results
            pso_coords, pso_fitness, pso_time = run_pso_optimization(california_polygon)
            ga_coords, ga_fitness, ga_time = run_ga_optimization(california_polygon)

            print(f"Best PSO Fitness: {pso_fitness:.4f} | GA Fitness: {ga_fitness:.4f}")
            print(f"PSO Time: {pso_time:.2f}s | GA Time: {ga_time:.2f}s")

            if ga_fitness < best_fitness:
                best_fitness = ga_fitness
                best_coords = ga_coords

            results.append({
                'Run': run_idx,
                'Best PSO Fitness': pso_fitness,
                'Best GA Fitness': ga_fitness,
                'Time (s) (PSO)': pso_time,
                'Time (s) (GA)': ga_time,
                'Coordinates': ga_coords
            })
        except ValueError as ve:
            print(f"Run #{run_idx} failed: {ve}")
            results.append({
                'Run': run_idx,
                'Best PSO Fitness': None,
                'Best GA Fitness': None,
                'Time (s) (PSO)': None,
                'Time (s) (GA)': None,
                'Coordinates': None
            })

    print("\n--- Best Run ---")
    print(f"Best Fitness: {best_fitness:.4f}")
    print(f"Best Coordinates: {best_coords}")

    # Save all results to CSV
    results_df = pd.DataFrame(results)
    print("\nAll Results:\n", results_df)
    results_df.to_csv("hybrid_results.csv", index=False)

    # Mapping the Best Coordinates using Folium
    if best_coords:
        map_center = [np.mean([c[0] for c in best_coords]), np.mean([c[1] for c in best_coords])]
        m = folium.Map(location=map_center, zoom_start=6)

        # Plot existing stations (blue)
        for lat, lon in existing_locations:
            folium.CircleMarker(
                location=[lat, lon],
                radius=3,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6,
                popup='Existing Station'
            ).add_to(m)

        # Plot GA new stations (red)
        for lat, lon in best_coords:
            folium.Marker(
                location=[lat, lon],
                popup=f'GA Station',
                icon=folium.Icon(color='red', icon='star')
            ).add_to(m)
            folium.Circle(
                location=[lat, lon],
                radius=threshold_distance * 1000,  # Convert km to meters
                color='red',
                fill=False,
                weight=1,
                opacity=0.5,
                tooltip=f'Threshold: {threshold_distance:.2f} km'
            ).add_to(m)

        # Highlight California Boundary
        folium.GeoJson(
            california.__geo_interface__,
            name="California",
            style_function=lambda x: {
                'fillColor': 'none',
                'color': 'green',
                'weight': 2
            }
        ).add_to(m)

        # Save the map
        m.save("hybrid_best_run_map.html")
        print("Folium map saved to 'hybrid_best_run_map.html'.")

if __name__ == "__main__":
    main()


Threshold distance: 11.77 km


c:\Users\chait\OneDrive\Desktop\EV RP\venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\chait\OneDrive\Desktop\EV RP\venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



--- Hybrid Simulation #1 ---
Stopping search: maximum iterations reached --> 50
Best PSO Fitness: 46.2454 | GA Fitness: 21.4554
PSO Time: 113.96s | GA Time: 205.59s

--- Hybrid Simulation #2 ---
Stopping search: maximum iterations reached --> 50
Best PSO Fitness: 73.1402 | GA Fitness: 34.9506
PSO Time: 140.40s | GA Time: 207.59s

--- Hybrid Simulation #3 ---
Stopping search: maximum iterations reached --> 50
Best PSO Fitness: 45.1017 | GA Fitness: 42.5158
PSO Time: 120.19s | GA Time: 209.64s

--- Hybrid Simulation #4 ---
Stopping search: maximum iterations reached --> 50
Best PSO Fitness: 1000000.0000 | GA Fitness: 48.1365
PSO Time: 41.46s | GA Time: 210.17s

--- Hybrid Simulation #5 ---
Stopping search: maximum iterations reached --> 50
Best PSO Fitness: 1000000.0000 | GA Fitness: 29.4616
PSO Time: 75.56s | GA Time: 211.96s

--- Hybrid Simulation #6 ---
Stopping search: maximum iterations reached --> 50
Best PSO Fitness: 1000000.0000 | GA Fitness: 42.6178
PSO Time: 41.86s | GA Time: 